In [196]:
import spotipy
import pandas as pd
import numpy as np
from spotipy.oauth2 import SpotifyClientCredentials, SpotifyOAuth

In [197]:
client_id='5f398b52c96b431b989c91ef5441858c'
client_secret='1a10dd78dc2d43609b4c0a2a047e327f'
redirect_uri='https://open.spotify.com/'
token_url = "https://accounts.spotify.com/api/token"

## Project Outline

- download a dataset of spotify playlists
- find or write a library to handle genre etc analysis
- group similar playlists
- use plotting techniques to show clusters?

Project Proposal:
I am personally interested in analyzing my longterm listening and curation history through Spotify and spotipy. I have an extensive collection of playlists, which due to their regularity, can be analyzed as objects. The intention of the analysis is to uncover trends in the groupings, visualize the relative distances (similarities) of songs within playlists, and create a prediction mechanism for most similar playlists. I will use sklearn, matplotlib, and seaborn for the clustering and visualizations.

## User Auth

In [237]:
scope = 'playlist-read-private'

sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id=client_id, client_secret=client_secret, redirect_uri=redirect_uri, scope=scope, open_browser=True))

In [238]:
# scope = 'playlist-read-private'

# oauth = spotipy.util.prompt_for_user_token(client_id=client_id, client_secret=client_secret, redirect_uri=redirect_uri, scope=scope)
# sp = spotipy.Spotify(auth=oauth)

In [239]:
if sp:
    print(sp.me())

{'display_name': 'amaya lim', 'external_urls': {'spotify': 'https://open.spotify.com/user/nightrainlily'}, 'href': 'https://api.spotify.com/v1/users/nightrainlily', 'id': 'nightrainlily', 'images': [{'url': 'https://i.scdn.co/image/ab67757000003b82180bd1d0c567a1e26710aeb8', 'height': 64, 'width': 64}, {'url': 'https://i.scdn.co/image/ab6775700000ee85180bd1d0c567a1e26710aeb8', 'height': 300, 'width': 300}], 'type': 'user', 'uri': 'spotify:user:nightrainlily', 'followers': {'href': None, 'total': 343}, 'country': 'US', 'product': 'premium', 'explicit_content': {'filter_enabled': False, 'filter_locked': False}, 'email': 'amayalim99@gmail.com'}


## Getting Playlists

In [207]:
def get_all_playlists():
    playlist_count = 1000
    results = []
    offset = 0
    while offset < playlist_count:
        results.append(sp.current_user_playlists(limit=50, offset=offset))
        offset += 50
    return results
results = get_all_playlists()
for result in results:
    for playlist in result['items']:
        print("Playlist Name:", playlist['name'])
        print("Playlist ID:", playlist['id'])

Playlist Name: 00 infinity
Playlist ID: 6P9UVbxRLT7p2IDDHbFZaD
Playlist Name: 835 maybe tomorrow
Playlist ID: 4cGtX4hh7a9LgGy5hhv41m
Playlist Name: 834 coming around
Playlist ID: 5Iw1dfOuejmqp8b2YUYMHf
Playlist Name: currently
Playlist ID: 4ataPSlIvLaL4HDHwVSOeS
Playlist Name: only what we feel //
Playlist ID: 0K3M3uuT3ckXDWpYVmmZZT
Playlist Name: 834 heartbreak
Playlist ID: 09gQw7M9fA7QjzygiqEgaH
Playlist Name: 833 silverlake
Playlist ID: 5i9v3QcICRGu9PLI2TjWKH
Playlist Name: 832 might as well
Playlist ID: 4P0idnT4X3TuYoKOBiBhIV
Playlist Name: 831 if it's forever
Playlist ID: 3vqj9cpxyibvZjZIeGS0aw
Playlist Name: 830 if i let you go
Playlist ID: 4BeJXlWQdA06fmHuvXqnD9
Playlist Name: 829 crush
Playlist ID: 5udjzyMIXiAgUi5OnstlQ1
Playlist Name: 828 more than i can give
Playlist ID: 6d3yzAwEF2l8xwQRRZbQGe
Playlist Name: 827 just fine
Playlist ID: 7s1bQRxm7tiTJO3WPydYR6
Playlist Name: 826 running in circles
Playlist ID: 73pXsKkjIiEq9e2FolBqBn
Playlist Name: 825 my mistake
Playlist ID: 6re

In [210]:
playlist_data = []

for result in results:
    for playlist in result['items']:
        playlist_data.append({
            'name' : playlist['name'],
            'track_ids' : sp.playlist_tracks(playlist['id'], fields='items(track(id))'),
            'track_names' : sp.playlist_tracks(playlist['id'], fields='items(track(name))'),
            'id' : playlist['id'],
            # 'image' : playlist['images'][0].get('url')
        })

playlists = pd.DataFrame(playlist_data)
# playlists.head().style.format({'image': lambda x : f'<img src="{x}" width="100px"/>'})
playlists.head()

,name,track_ids,track_names,id
0,00 infinity,{'items': [{'track': {'id': '5UvgTF3oGUxRwi96U...,{'items': [{'track': {'name': 'symbol'}}]},6P9UVbxRLT7p2IDDHbFZaD
1,835 maybe tomorrow,{'items': [{'track': {'id': '7BRD7x5pt8Lqa1eGY...,"{'items': [{'track': {'name': 'CHIHIRO'}}, {'t...",4cGtX4hh7a9LgGy5hhv41m
2,834 coming around,{'items': [{'track': {'id': '6I9GeNnqNqQ5Q8DSr...,{'items': [{'track': {'name': 'Method Actor'}}...,5Iw1dfOuejmqp8b2YUYMHf
3,currently,{'items': [{'track': {'id': '5N3hjp1WNayUPZrA8...,{'items': [{'track': {'name': 'Please Please P...,4ataPSlIvLaL4HDHwVSOeS
4,only what we feel //,{'items': [{'track': {'id': '2DBfUucA9WyTrbZOC...,{'items': [{'track': {'name': 'On the Way Home...,0K3M3uuT3ckXDWpYVmmZZT


In [211]:
def pretty_tracks(col, name):
        all_l = []
        for i in np.arange(len(playlists[col])):
                row = playlists[col].values[i]
                l = []
                for j in row:
                        item = row['items']
                        for k in np.arange(len(item)):
                                track = item[k]['track']
                                id = track[name]
                                l.append(id)
                all_l.append(l)
        return all_l

playlists['track_ids'] = pretty_tracks('track_ids', 'id')
playlists['track_names'] = pretty_tracks('track_names', 'name')

playlists.head().style.format({'image': lambda x : f'<img src="{x}" width="100px"/>'})


,name,track_ids,track_names,id
0,00 infinity,['5UvgTF3oGUxRwi96UZJd4I'],['symbol'],6P9UVbxRLT7p2IDDHbFZaD
1,835 maybe tomorrow,"['7BRD7x5pt8Lqa1eGYC4dzj', '2IWtloZYQDcP8Ashwx8QEF', '4rmVZajAF7PkrCagGPHbqa', '2FYGZDfsAnNsrm1gVbyKnG', '5c9tBmJKbTdn1vhzXHeAwW', '1nzOUWyHoZLzf9EX0pnFPn', '4KW1lqgSr8TKrvBII0Brf8', '0YQEWdfq3ajtNtK106Dr0k', '1n4jwRVXdkK2U34nBDUKKT']","['CHIHIRO', 'Black Out Days', 'Sticky', 'SAOKO', 'Sympathy is a knife', 'leavemealone', 'Father Stretch My Hands Pt. 1', 'Shinigami Eyes', 'BLOOD.']",4cGtX4hh7a9LgGy5hhv41m
2,834 coming around,"['6I9GeNnqNqQ5Q8DSrFkkvd', '0M7oki70AV10Ztt3HyUd8Z', '15uPQvxN5wjdpO8THuhuNP']","['Method Actor', 'Ladies Room', 'Good Stuff']",5Iw1dfOuejmqp8b2YUYMHf
3,currently,"['5N3hjp1WNayUPZrA8kJmJP', '1MYWmuOGiw6lHuEYZM9pdP', '6N3mmEBqDmRhYNH8UN9QLs', '6t3RJKJh8DYIaYh82Xy2LJ', '2LZ2CNbcbJrwh53pitRgom', '3OaFGqHUyxGVkOcSILw8Kx', '1WH2bOLCi7xwv1w6tvSojq', '6YtylNU7D1S5SeO529q2AN', '3SYNTZgVt7Wht3jRf0ElGh', '3QWLuJZ2A5TJ1BN6H0AMg7', '6uvO7oNgWUBR7t4W9Jymq1', '6hp729YavhhHeH62B2QH6J', '5g7Ppc6qpnkRpF5CMSpxn8', '5FV3ZxWtkEr89svQq3rFxI', '6ugo6TrO93ZW9gAovTzegl', '4fWi9tlWzs2XUejI8qjC6d', '29b70iOlgRkYQ8ZzINZXph', '5A9scs52hDGefDOTRNio5o']","['Please Please Please', 'Elaine', 'Hasta La Vista', 'Faded', 'Silver Soul', 'I think about it all the time', 'Swimming Big', 'Evergreen', 'Oh, Gemini', 'Vaughn', 'Empty Vessel', 'Wall Of Eyes', 'Birdseye', 'Lost on Me', 'How Long', ""It'll All Work Out"", 'Sad Eyes - Studio Outtake - 1990', 'Light of a Clear Blue Morning']",4ataPSlIvLaL4HDHwVSOeS
4,only what we feel //,"['2DBfUucA9WyTrbZOCK3j4v', '5qTc3VX4ibhIBWsEV82LA1']","['On the Way Home - Live at Massey Hall 1971', 'That Song About the Midway']",0K3M3uuT3ckXDWpYVmmZZT


## Getting Song Data

In [215]:
songs = playlists
songs['zipp'] = list(zip(songs['track_ids'], songs['track_names']))
songs['zipp'][0]

(['5UvgTF3oGUxRwi96UZJd4I'], ['symbol'])

In [216]:
def merge(z):
    return list(tuple(zip(z[0], z[1])))

songs['zipp'] = songs['zipp'].apply(merge)
songs.head()

,name,track_ids,track_names,id,zipp
0,00 infinity,[5UvgTF3oGUxRwi96UZJd4I],[symbol],6P9UVbxRLT7p2IDDHbFZaD,"[(5UvgTF3oGUxRwi96UZJd4I, symbol)]"
1,835 maybe tomorrow,"[7BRD7x5pt8Lqa1eGYC4dzj, 2IWtloZYQDcP8Ashwx8QE...","[CHIHIRO, Black Out Days, Sticky, SAOKO, Sympa...",4cGtX4hh7a9LgGy5hhv41m,"[(7BRD7x5pt8Lqa1eGYC4dzj, CHIHIRO), (2IWtloZYQ..."
2,834 coming around,"[6I9GeNnqNqQ5Q8DSrFkkvd, 0M7oki70AV10Ztt3HyUd8...","[Method Actor, Ladies Room, Good Stuff]",5Iw1dfOuejmqp8b2YUYMHf,"[(6I9GeNnqNqQ5Q8DSrFkkvd, Method Actor), (0M7o..."
3,currently,"[5N3hjp1WNayUPZrA8kJmJP, 1MYWmuOGiw6lHuEYZM9pd...","[Please Please Please, Elaine, Hasta La Vista,...",4ataPSlIvLaL4HDHwVSOeS,"[(5N3hjp1WNayUPZrA8kJmJP, Please Please Please..."
4,only what we feel //,"[2DBfUucA9WyTrbZOCK3j4v, 5qTc3VX4ibhIBWsEV82LA1]","[On the Way Home - Live at Massey Hall 1971, T...",0K3M3uuT3ckXDWpYVmmZZT,"[(2DBfUucA9WyTrbZOCK3j4v, On the Way Home - Li..."


In [217]:
songs = songs.explode('zipp')
songs.head()

,name,track_ids,track_names,id,zipp
0,00 infinity,[5UvgTF3oGUxRwi96UZJd4I],[symbol],6P9UVbxRLT7p2IDDHbFZaD,"(5UvgTF3oGUxRwi96UZJd4I, symbol)"
1,835 maybe tomorrow,"[7BRD7x5pt8Lqa1eGYC4dzj, 2IWtloZYQDcP8Ashwx8QE...","[CHIHIRO, Black Out Days, Sticky, SAOKO, Sympa...",4cGtX4hh7a9LgGy5hhv41m,"(7BRD7x5pt8Lqa1eGYC4dzj, CHIHIRO)"
1,835 maybe tomorrow,"[7BRD7x5pt8Lqa1eGYC4dzj, 2IWtloZYQDcP8Ashwx8QE...","[CHIHIRO, Black Out Days, Sticky, SAOKO, Sympa...",4cGtX4hh7a9LgGy5hhv41m,"(2IWtloZYQDcP8Ashwx8QEF, Black Out Days)"
1,835 maybe tomorrow,"[7BRD7x5pt8Lqa1eGYC4dzj, 2IWtloZYQDcP8Ashwx8QE...","[CHIHIRO, Black Out Days, Sticky, SAOKO, Sympa...",4cGtX4hh7a9LgGy5hhv41m,"(4rmVZajAF7PkrCagGPHbqa, Sticky)"
1,835 maybe tomorrow,"[7BRD7x5pt8Lqa1eGYC4dzj, 2IWtloZYQDcP8Ashwx8QE...","[CHIHIRO, Black Out Days, Sticky, SAOKO, Sympa...",4cGtX4hh7a9LgGy5hhv41m,"(2FYGZDfsAnNsrm1gVbyKnG, SAOKO)"


In [218]:
songs['track_name'] = songs['zipp'].map(lambda x: x[1])
songs['track_id'] = songs['zipp'].map(lambda x: x[0])
songs.head()

,name,track_ids,track_names,id,zipp,track_name,track_id
0,00 infinity,[5UvgTF3oGUxRwi96UZJd4I],[symbol],6P9UVbxRLT7p2IDDHbFZaD,"(5UvgTF3oGUxRwi96UZJd4I, symbol)",symbol,5UvgTF3oGUxRwi96UZJd4I
1,835 maybe tomorrow,"[7BRD7x5pt8Lqa1eGYC4dzj, 2IWtloZYQDcP8Ashwx8QE...","[CHIHIRO, Black Out Days, Sticky, SAOKO, Sympa...",4cGtX4hh7a9LgGy5hhv41m,"(7BRD7x5pt8Lqa1eGYC4dzj, CHIHIRO)",CHIHIRO,7BRD7x5pt8Lqa1eGYC4dzj
1,835 maybe tomorrow,"[7BRD7x5pt8Lqa1eGYC4dzj, 2IWtloZYQDcP8Ashwx8QE...","[CHIHIRO, Black Out Days, Sticky, SAOKO, Sympa...",4cGtX4hh7a9LgGy5hhv41m,"(2IWtloZYQDcP8Ashwx8QEF, Black Out Days)",Black Out Days,2IWtloZYQDcP8Ashwx8QEF
1,835 maybe tomorrow,"[7BRD7x5pt8Lqa1eGYC4dzj, 2IWtloZYQDcP8Ashwx8QE...","[CHIHIRO, Black Out Days, Sticky, SAOKO, Sympa...",4cGtX4hh7a9LgGy5hhv41m,"(4rmVZajAF7PkrCagGPHbqa, Sticky)",Sticky,4rmVZajAF7PkrCagGPHbqa
1,835 maybe tomorrow,"[7BRD7x5pt8Lqa1eGYC4dzj, 2IWtloZYQDcP8Ashwx8QE...","[CHIHIRO, Black Out Days, Sticky, SAOKO, Sympa...",4cGtX4hh7a9LgGy5hhv41m,"(2FYGZDfsAnNsrm1gVbyKnG, SAOKO)",SAOKO,2FYGZDfsAnNsrm1gVbyKnG


In [223]:
songs.reset_index()

,index,name,track_ids,track_names,id,zipp,track_name,track_id
0,0,00 infinity,[5UvgTF3oGUxRwi96UZJd4I],[symbol],6P9UVbxRLT7p2IDDHbFZaD,"(5UvgTF3oGUxRwi96UZJd4I, symbol)",symbol,5UvgTF3oGUxRwi96UZJd4I
1,1,835 maybe tomorrow,"[7BRD7x5pt8Lqa1eGYC4dzj, 2IWtloZYQDcP8Ashwx8QE...","[CHIHIRO, Black Out Days, Sticky, SAOKO, Sympa...",4cGtX4hh7a9LgGy5hhv41m,"(7BRD7x5pt8Lqa1eGYC4dzj, CHIHIRO)",CHIHIRO,7BRD7x5pt8Lqa1eGYC4dzj
2,1,835 maybe tomorrow,"[7BRD7x5pt8Lqa1eGYC4dzj, 2IWtloZYQDcP8Ashwx8QE...","[CHIHIRO, Black Out Days, Sticky, SAOKO, Sympa...",4cGtX4hh7a9LgGy5hhv41m,"(2IWtloZYQDcP8Ashwx8QEF, Black Out Days)",Black Out Days,2IWtloZYQDcP8Ashwx8QEF
3,1,835 maybe tomorrow,"[7BRD7x5pt8Lqa1eGYC4dzj, 2IWtloZYQDcP8Ashwx8QE...","[CHIHIRO, Black Out Days, Sticky, SAOKO, Sympa...",4cGtX4hh7a9LgGy5hhv41m,"(4rmVZajAF7PkrCagGPHbqa, Sticky)",Sticky,4rmVZajAF7PkrCagGPHbqa
4,1,835 maybe tomorrow,"[7BRD7x5pt8Lqa1eGYC4dzj, 2IWtloZYQDcP8Ashwx8QE...","[CHIHIRO, Black Out Days, Sticky, SAOKO, Sympa...",4cGtX4hh7a9LgGy5hhv41m,"(2FYGZDfsAnNsrm1gVbyKnG, SAOKO)",SAOKO,2FYGZDfsAnNsrm1gVbyKnG
...,...,...,...,...,...,...,...,...
11928,999,008 a place we've never been before // fast gl...,"[3HFBqhotJeEKHJzMEW31jZ, 1LqqMolbCWESuXG9mu97o...","[Change, Chinese Apple, From The Morning, Risi...",1sb3iJhU0XESXUsURIRxk1,"(7kZLYVrNWJje4Iws6Zv3Fe, Bring Me)",Bring Me,7kZLYVrNWJje4Iws6Zv3Fe
11929,999,008 a place we've never been before // fast gl...,"[3HFBqhotJeEKHJzMEW31jZ, 1LqqMolbCWESuXG9mu97o...","[Change, Chinese Apple, From The Morning, Risi...",1sb3iJhU0XESXUsURIRxk1,"(3GC9zYBGiOWnv6Crx6BOUA, Often)",Often,3GC9zYBGiOWnv6Crx6BOUA
11930,999,008 a place we've never been before // fast gl...,"[3HFBqhotJeEKHJzMEW31jZ, 1LqqMolbCWESuXG9mu97o...","[Change, Chinese Apple, From The Morning, Risi...",1sb3iJhU0XESXUsURIRxk1,"(7hcTkpQKdrTnK9zJijeZVP, Polyurethane)",Polyurethane,7hcTkpQKdrTnK9zJijeZVP
11931,999,008 a place we've never been before // fast gl...,"[3HFBqhotJeEKHJzMEW31jZ, 1LqqMolbCWESuXG9mu97o...","[Change, Chinese Apple, From The Morning, Risi...",1sb3iJhU0XESXUsURIRxk1,"(4uyTw118ri02rxRcaF6vnz, Only Mystery)",Only Mystery,4uyTw118ri02rxRcaF6vnz


### Audio Features

In [224]:
audio_features_results = songs['track_id'].apply(sp.audio_features)
audio_features_results

Max Retries reached


SpotifyException: http status: 429, code:-1 - /v1/audio-features/?ids=5UvgTF3oGUxRwi96UZJd4I:
 Max Retries, reason: too many 429 error responses